![title](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fdyclassroom.files.wordpress.com%2F2019%2F09%2Fmongodb.jpg&f=1&nofb=1)

<div style="background-color: #168b33; padding: 5px;"></div>

# Unstrukturierte Daten aus einer MongoDB auslesen und auswerten


1. [Übersicht](#Uebersicht)
    1. [Einführung in die dokumentenorientierte MongoDB](#Einführung)
    2. [Ziel dieses Notebooks](#Ziel)
2. [MongoDB Grundlagen](#Grundlagen)
    1. [JSON und BSON](#BSON)
    2. [Relationships with Documents](#Relationships)
    3. [Aufgabenbereich 1](#Aufgabenbereich1)
3. [Starten der MongoDB und Verbdindung herstellen](#Start)
    1. [Lesen, speichern, updaten und löschen von Daten](#CRUD)
    2. [Aufgabenbereich 2](#Aufgabenbereich2)
3. [Betrachten des Datenbestandes](#Datenbestand)
    1. [Importieren eines JSON Datensatzes](#JSON-Einlesen)
    2. [Datensatz mit Pandas anzeigen](#Pandas)
    3. [Die Dokumente direkt auslesen und mit Pandas anzeigen](#Direkt)
    4. [Aufgabenbereich 3](#Aufgabenbereich3)
5. [Erstellen von Queries mit den MongoDB Operatoren](#Queries)
    1. [Übersicht über die Operatoren](#Operatoren-Uebersicht)
    2. [Beispiele für die Operatoren](#beispiele-operatoren)
    3. [Aufgabenbereich 4](#Aufgabenbereich4)
6. [Auslesen und Zusammenhänge herausfinden mit der Aggregation Pipeline](#Pipeline)
    1. [Unterschied zwischen Operatoren und Stages](#Pipeline-Stages-Operatoren)
    2. [Implementieren einer Pipeline (Syntax)](#Pipeline-Syntax)
    3. [Beispiele mit der Aggregation Pipeline](#Pipeline-Beispiele) 
    4. [Aufgabenbereich 5](#Aufgabenbereich5)
7. [Auslesen und Zusammenhänge herausfinden mit dem Map-Reduce Verfahren](#Map)
    1. [Aufgabenbereich 6](#Aufgabenbereich6)
8. [Abschlussaufgabe](#Abschlussaufgabe)
    1. [Aufgabenbereich 7](#Aufgabenbereich7)

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 1. <b id="Uebersicht">Übersicht</b>

### 1.1 <b id="Einführung">Einführung in die dokumentenorientierte MongoDB</b>
Große Datenmengen werden heutzutage schnell generiert. Die MongoDB ist eine dokumentenorientierte Datenbank, die oft dazu verwendet wird um strukturlose Daten in einem verteilten Cluster aus Datenbanken abzuspeichern. Einige Eigenschaften der MongoDB stechen besonders hervor, wenn es um die Wahl des geeigneten Database Management Systems
geht. Hierzu zählen eine richtige Abfragesprache für dynamische Abfragen, eine hohe Verfügbarkeit, Skalierbarkeit, eine effiziente Speicher Engine, Datenkonsistenz und Datenintegrität.

MongoDB eignet sich am besten für große Datenmengen, wenn die vorhandenen Daten weitere Manipulationen für die gewünschte Ausgabe benötigen. Zu den leistungsstarken Funktion zählen hierbei die CRUD-Operationen, das Aggregations-Framework, die Textsuche und die Map-Reduce-Funktion.

### 1.2 Ziel dieses Notebooks
Es soll verdeutlicht werden wie aus einer NoSQL Datenbank Daten effizient ausgelesen werden können, um diese zu bearbeiten. Datensätze liegen nicht immer als CSV Datei vor oder liegen in einem verteilten System aus Datenbanken. Diese Daten direkt aus der Datenbank auszulesen und mit effektiven Werkzeugen wie dem Aggregations-Framework zu strukturieren ist das Ziel dieses Notebooks.

<b>Setup für die Aufgabenüberprüfung</b>

In [100]:
from taskreview.learningModule import LearningModule
mongo_lm = LearningModule('mongoData/MongoTask.db')

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 2. <b id="Grundlagen">Grundlagen</b>

Es sollen in diesem Kapitel die Grundlagen für die Interaktion mit der dokumentenorientierten Datenbank MongoDB und die Verwendung der JavaScript Object Notation (JSON) erklärt werden. Diese ist wichtig, denn die MongoDB speichert Daten nicht in Tabellen, sondern in Collections, welche keine festen Strukturen enthalten, ab. Die Collections können als äquivalent zu den Tabellen in SQL Datenbanken angesehen werden. Die Dokumente selbst liegen innerhalb dieser Collections und werden in einer erweiterten JSON Struktur dargestellt, welche aber mehr Datentypen bietet. Diese erweiterte JSON Struktur wird als Binary JSON (BSON) bezeichnet.

### 2.1 <b id="BSON">JSON und BSON</b>
Der Aufbau der Datensätze wird hier genauer betrachtet. Die Struktur eines einzelnen Dokumentes ist die gleiche wie die von JSON Dateien. In der unteren Abbildung ist ein Dokument zu sehen welches in einer MongoDB gespeichert wurde. Die Daten werden als Key/Value Paare dargestellt, wobei beim ersten Paar mit dem Key "_id" zu erkennen ist, dass der Value einen neuen Datentyp enthällt.

![title](img/image_json.png)

Die neuen Datentypen werden als BSON Datentypen bezeichnet. Die MongoDB speichert die Datentypen in einer binären Form ab, wodurch es möglich ist die JSON Datentypen zu ergänzen. Der Key "_id" enthällt somit den BSON Datentyp ObjectId(). Dieses Feld repräsentiert die einzigartige ID jedes Dokumentes. Diese wird benötigt um ein Dokument zu identifizieren (vergleichbar mit Primärschlüssel) und wird von der MongoDB automatisch generiert. Es ist aber nicht zwingend Notwendig den Primärschlüssel als ObjectID() zu definierten. Selbstdefinierte Schlüssel aus Strings, Integern oder anderen Komibnationen sind ebenfalls zulässig.

<table style="font-size:14px">
	<tbody>
        <tr>
			<td style="width: 200px;text-align: center;" align="center"><b>BSON Datentypen</b></td>
			<td style="width: 200px;text-align: center;" align="center"><b>JSON Datentypen</b></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Double</td>
			<td style="width: 200px;text-align: center;" align="center">Number</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">String</td>
			<td style="width: 200px;text-align: center;" align="center">String</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Object</td>
			<td style="width: 200px;text-align: center;" align="center">Object</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Array</td>
			<td style="width: 200px;text-align: center;" align="center">Array</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Boolean</td>
			<td style="width: 200px;text-align: center;" align="center">Boolean</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Null</td>
			<td style="width: 200px;text-align: center;" align="center">Null/Empty</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Binary data</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Undefined</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">ObjectId</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Date</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Regular Expression</td>
			<td style="width: 200px;text-align: center;" align="center"> </td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">JavaScript</td>
			<td style="width: 200px;text-align: center;" align="center"></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Symbol</td>
			<td style="width: 200px;text-align: center;" align="center"></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">32-bit integer</td>
			<td style="width: 200px;text-align: center;" align="center"></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">64-bit integer</td>
			<td style="width: 200px;text-align: center;" align="center"></td>
		</tr>
	</tbody>
</table>

### 2.2 <b id="Relationships">Relationships with Documents</b>
Die MongoDB kann logische Beziehungen zwischen Dokumenten abbilden. Dazu gibt es verschiedene Verfahren die hier genauer erläutert werden sollen.  

<b>One-to-One Relationships with Embedded Documents</b><br>
Ein Dokument kann als Key den Datentyp "Object" enthalten. Dies repräsentiert bei der MongoDB ein Embedded Document. Es wird somit ein gesamtes Dokument unterhalb eines einzelnen Keys eingetragen.

![title](img/image_json_embedded.png)

<b>One-to-Many with Document References</b><br>
Ein einzelner Key kann mehrere Primärschlüssel in einem Array enthalten, welche dann zu vielen weiteren Dokumenten führen.

![title](img/image_json_references.png)

<b>One-to-Many Relationships with Embedded Documents</b><br>
In einem Array können Objekte als Datentypen eingefügt werden. Ein Key kann somit viele einzelne Dokumente in einem Array enthalten. Der Key des Arrays verweist somit auf viele einzelne Dokumente.

![title](img/image_json_many_embedded.png)

<div style="background-color:  #168b33; padding: 5px;height: 15px;"></div>

### 2.3 <b id="Aufgabenbereich1">Aufgabenbereich 1</b>

Wie heißt der Datentyp für den Primärschlüssel aus dem Dokument das bei den Grundlagen gezeigt wurde?

In [ ]:
mongo_lm.show_task(1)

Output()

Output()

Output()

In welcher Form werden die Daten in der MongoDB gespeichert?

In [ ]:
mongo_lm.show_task(2)

Output()

Output()

Output()

Wie heißt die erweiterte JSON-Form die in der MongoDB intern verwendet wird? 

In [4]:
mongo_lm.show_task(3)

Output()

Output()

Output()

Markiere das Dokument welches ein Embedded Document enthält:

In [5]:
mongo_lm.show_task(4)

Output()

Output()

Output()



Im weiteren Schritt wird gezeigt wie die MongoDB gestartet wird und erste Datensätze importiert werden können. 

Anhand dieser Daten sollen die Grundladen zu den CRUD Operationen verdeutlicht werden. CRUD steht hierbei für die vier Interaktionsmöglichkeiten wenn es darum geht Dokumente zu erstellen (create), lesen (read), aktualisieren (update) und zu löschen (delete).

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 3 <b id="Start">Starten der MongoDB und Verbdindung herstellen</b>

In [6]:
#!mongod --config /home/jovyan/mongod.conf

<b>MongoDB Treiber importieren</b>

Hier werden alle notwendigen Module für die Interaktion mit der MongoDB und den Daten geladen.

In [9]:
import pymongo
import datetime
import json 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bson.son import SON
from bson.objectid import ObjectId
from pprint import pprint

<table style="font-size:14px">
	<tbody >
        <tr>
            <td style="width: 200px;text-align: center;" align="center"><b>Modul</b></td>
            <td style="width: 200px;text-align: center;" align="center"><b>Verwendung</b></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">pymongo</td>
			<td style="width: 200px;text-align: center;" align="center">MongoDB Python Treiber</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">datetime</td>
			<td style="width: 200px;text-align: center;" align="center">Ein neues Datum erstellen</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">json</td>
			<td style="width: 200px;text-align: center;" align="center">JSON encoder and decoder</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">matplotlib</td>
			<td style="width: 200px;text-align: center;" align="center">Visualization with Python</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">pandas</td>
			<td style="width: 200px;text-align: center;" align="center">Daten-Manipulation und -Analyse</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">numpy</td>
			<td style="width: 200px;text-align: center;" align="center">Mathematische Funktionen</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">SON (Serialized Ocument Notation)</td>
			<td style="width: 200px;text-align: center;" align="center">Interaktion mit den BSON Documenten</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">pprint</td>
			<td style="width: 200px;text-align: center;" align="center">Data pretty printer</td>
		</tr>
	</tbody>
</table>

<b>Verbindung zu MongoDB herstellen und alle vorhandenen Datenbanken ausgeben</b></br>

Die MondoDB enhtällt 3 default Datenbanken. Die <b>local</b> Datenbank welche Daten der lokalen Instanz beinhaltet wie z.B. Metadaten. Die <b>admin</b> Database welche die Authentifikation mit der jeweiligen Instanz ermöglicht. Die <b>config</b> Datenbank welche die Informationen über die Verbindung und die mögliche Anbindung an ein Cluster enhällt. 

In [11]:
client = pymongo.MongoClient()
client.list_database_names()

['Store', 'admin', 'config', 'local']

<b>Eine neue Datenbank und eine Collection anlegen<b>

In [12]:
store_db = client["Store"]
user_col = store_db["User"]

### 3.1 <b id="CRUD">Lesen, speichern, updaten und löschen von Daten</b>


<b>Syntax der CRUD-Operationen</b>

![title](img/syntax_crud.jpg)

<b>Ein Dokument anlegen und in der Collection abspeichern</b></br>

<b>Bemerke: </b>Das Dokument enthält keine ObjectId, da diese von der MongoDB selbst generiert wird.

In [74]:
example = {
    "first_name" : "Neuer Name",
    "last_name" : "Neuer Nachname",
    "email" : "irgendwas@irgendwas",
    "phone_number" : "+55 744 995 7575",
    "last_login" : datetime.datetime.now(),
    "address" : {
        "country" : "A Country",
        "city" : "A City",
        "street" : "A Street",
        "zip" : 99999
    },
    "cart" : [] 
}
pprint(example)

{'address': {'city': 'A City',
             'country': 'A Country',
             'street': 'A Street',
             'zip': 99999},
 'cart': [],
 'email': 'irgendwas@irgendwas',
 'first_name': 'Neuer Name',
 'last_login': datetime.datetime(2021, 1, 8, 13, 31, 44, 283794),
 'last_name': 'Neuer Nachname',
 'phone_number': '+55 744 995 7575'}


In [75]:
# Wir speichern die ID um das Dokument in der Datenbank wiederzufinden
id = user_col.insert_one(example).inserted_id
pprint(id)

ObjectId('5ff850b294d1268f22add640')


<b>Ein Dokument anhand eines Filters auslesen</b>


Zum auslesen eines Dokumentes muss durch das <b>MongoCursor</b> Objekt iteriert werden.<br>
<b style="color:red">Achtung:</b> Wird eine Cursor einmal ausgelesen ist das Object leer.

In [ ]:
# Wir erstellen ein Query-Dokument um ein MongoCursor-Objekt zu erstellen
query = {'_id': id}

inserted_Document = user_col.find(query)
# copy, da der Cursor sonst leer wäre
copy_document = inserted_Document.clone()
print(type(inserted_Document))

Dokumente müssen immer in einer <b>Iteration</b> aus einem MongoCursor ausgelesen werden

In [ ]:
for document in inserted_Document:
    pprint(document)

In einem <b>Pandas-DataFrame</b> lassen sich die Daten besser betrachten

In [ ]:
df = pd.DataFrame(copy_document)
df

<b>Updaten eines Dokumentes anhand eines Filters und dem Set Operator</b>

Operatoren wie <b>"$set"</b> werden von der MongoDB verwendet um Audrücke, Vergleiche oder Filter zu generieren. Die Operatoren werden in Kapitel 5 noch genauer betrachtet.

In [ ]:
# Die ID des Dokumentes das Sie voher erstellt hatten
query = {'_id': id} 
newvalue = { "$set": { 'first_name': "Peter Müller" } } 
inserted_Document = user_col.update_one(query,newvalue)

# Suche nach dem Dokument wiederholen um die Veränderung zu sehen
query = {'_id': id}
inserted_Document = user_col.find(query)
copy_document = inserted_Document.clone()

#Anzeigen als Dokument
for document in inserted_Document:
    pprint(document)

#Anzeigen in Pandas
df = pd.DataFrame(copy_document)
df

<b>Nach mehreren Dokumenten suchen</b>

Der <b>MongoCursor</b> enthällt zudem Operatoren die Direkt beim erstellen des Cursors angehängt werden können. Im folgenden Beispiel werden mit der <b>find()</b> Methode noch eine <b>sort()</b> und eine <b>limit()</b> Methode angehängt.

In [ ]:
for document in user_col.find().sort("_id",pymongo.ASCENDING).limit(1):
  pprint(document) 

<div style="background-color:  #168b33; padding: 5px;height: 15px;"></div>

### 3.2 <b id="Aufgabenbereich2">Aufgabenbereich 2</b>

Warum muss die ObjectId bei einem neuen Dokument nicht angegeben werden?

In [ ]:
mongo_lm.show_task(6)

Was passiert wenn der Cursor einmal ausgegeben wurde?

In [ ]:
mongo_lm.show_task(5)

Erstellen Sie ein neues Dokument und fügen Sie es in die Collection "Customer" ein. Folgende Daten sollen enthalten sein:
<ol>
    <li>  <b>_id :</b> 775 </li>
    <li>  <b>first_name :</b>Manuel</li>
    <li>  <b>last_name :</b>Mustermann</li>
    <li>  <b>email :</b>Manuel@Mustermann.de</li>
    <li>  <b>phone_number :</b>+49 125 145 1258</li>
    <li>  <b>last_login :</b>2011-11-09</li>
    <li>  <b>registration_date :</b> 2010-10-09</li>
</ol>

Fragen Sie ihr neu erstelltes Dokument anschließend an

In [ ]:
user_col = store_db["User"]
new_customer = {
    "_id":775,
    "first_name" : "Manuel",
    "last_name" : "Mustermann",
    "email" : "Manuel@Mustermann.de",
    "phone_number" : "+49 125 145 1258",
    "last_login" : datetime.datetime.fromisoformat('2019-12-04'),
    "registration_date" : datetime.datetime.fromisoformat("2010-10-09")
}

#user_col.insert_one(new_customer)

documents = user_col.find()
#Anzeigen in Pandas
df = pd.DataFrame(documents)
#Alle Datentypen im df werden für die Aufgabenüberprüfung in Strings umgewandelt
df = df.astype(str)
mongo_lm.show_task(100,df)

In [ ]:
Warum konnte das eben erstellte Dokument eingefügt werden obwohl das Beispiel das vor dem Aufgabenbereich 2 gezeigt wurde, noch die Felder "address" und "cart" enthält?

In [ ]:
mongo_lm.show_task(7)

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 4. <b id="Datenbestand">Betrachten des Datenbestandes</b>

Es sollen nun bereits vordefinierte Daten in die MongoDB importiert und betrachtet werden.

### 4.1 <b id="JSON-Einlesen">Importieren eines JSON Datensatzes</b>
Die lokalen Daten werden in die MongoDB importiert und in zwei unterschiedliche Collections abgelegt.

In [13]:
grocery_col = store_db["Grocery"]

In [ ]:
#!mongoimport --db Store --collection Grocery --file mongoData/grocery.json --jsonArray

In [14]:
customer_col = store_db["Customer"]

In [ ]:
#!mongoimport --db Store --collection Customer --file mongoData/customer.json --jsonArray

### 4.2 <b id="Pandas">Datensatz mit Pandas anzeigen</b>

Zuerst werden wir die Daten direkt aus dem JSON Dokument, welches lokal in diesem System hinterlegt ist, betrachten. Dieses Dokument wurde aus der MongoDB exportiert und enthällt alle vorhandenen Daten aus der Customer Collection.

In [15]:
df = pd.DataFrame(pd.read_json("mongoData/customer.json"))
df.head()

,_id,first_name,last_name,email,last_login,registration_date,address,cart,phone_number
0,{'$oid': '5fe60bb2fc13ae64ea000064'},Neely,Matskiv,nmatskiv0@abc.net.au,{'$date': '2020-06-24T22:00:00.000Z'},{'$date': '2020-06-24T22:00:00.000Z'},"{'country': 'Canada', 'city': 'Barrhead', 'str...","[{'$oid': '5fe6fc8ba789e6e217ef8a36'}, {'$oid'...",NaN
1,{'$oid': '5fe60bb2fc13ae64ea000065'},Hilde,Brownsett,hbrownsett1@mediafire.com,{'$date': '2019-08-01T22:00:00.000Z'},{'$date': '2019-08-01T22:00:00.000Z'},"{'country': 'Indonesia', 'city': 'Suka Makmue'...","[{'$oid': '5fe6fc8ba789e6e217ef88bf'}, {'$oid'...",NaN
2,{'$oid': '5fe60bb2fc13ae64ea000066'},Bob,Sheridan,bsheridan2@photobucket.com,{'$date': '2020-07-20T22:00:00.000Z'},{'$date': '2020-07-20T22:00:00.000Z'},"{'country': 'Netherlands', 'city': 'Haarlem', ...","[{'$oid': '5fe6fc8ba789e6e217ef89a0'}, {'$oid'...",NaN
3,{'$oid': '5fe60bb2fc13ae64ea000067'},Wait,Domek,wdomek3@surveymonkey.com,{'$date': '2019-06-18T22:00:00.000Z'},{'$date': '2019-06-18T22:00:00.000Z'},"{'country': 'Brazil', 'city': 'São Miguel dos ...","[{'$oid': '5fe6fc8ba789e6e217ef88d1'}, {'$oid'...",+55 744 995 7575
4,{'$oid': '5fe60bb2fc13ae64ea000068'},Bryn,Legat,blegat4@google.co.jp,{'$date': '2020-08-06T22:00:00.000Z'},{'$date': '2020-08-06T22:00:00.000Z'},"{'country': 'Indonesia', 'city': 'Muke', 'stre...","[{'$oid': '5fe6fc8ba789e6e217ef8764'}, {'$oid'...",+62 810 708 7339


Pandas kann den Datensatz nicht wie die MongoDB darstellen. Der Grund dafür ist das die MongoDB die BSON Datentypen im JSON Format als ein Embedded Document dargestellt werden. Beispiel ist "$oid" welches den Datentyp ObjectId repräsentiert. Die MongoDB verwendet diese Struktur um beim Import die Daten wieder in die ursprünglichen Datentypen einzufügen. Aus einem normalen JSON-Dokument könnten nur die JSON-Datentypen ausgelesen werden.

Die MongoDB  ist auch in der Lage die Daten als CSV Datensatz zu exportieren, aber dafür müssen beim Export Befehl in der Mongo Shell alle notwendigen Felder angegeben werden. Da jedes Dokument eine andere Struktur haben kann muss dieser Befehl abgeändert werden, wenn sich ein einzelnes Dokument um ein Feld vergrößert.

<b>Vorteil beim direkten auslesen aus der Datenbank: </b> Die Daten können live betrachtet, aktualisiert und bearbeitet werden. Aktualisierungen sind sofort sichtbar und es müssen keine zusätzlichen Dateien exportiert werden.

### 4.3 <b id="Direkt">Die Dokumente direkt auslesen und mit Pandas anzeigen</b>

Die Daten die in die MongoDB importiert worden sind, werden nun direkt aus der Collection ausgelesen und angezeigt. Um dies zu ermöglichen wird ein Object der Klassse MongoCursor angelegt welches auf die Dokumente verweist.

In [16]:
mongo_json = grocery_col.find({})

In [17]:
df = pd.DataFrame.from_records(mongo_json)
df

,_id,product,price,in_stock,sold,stock
0,5fe6fc8ba789e6e217ef870f,Pastry - Plain Baked Croissant,€4.39,True,95,76.0
1,5fe6fc8ba789e6e217ef8710,Scallops - U - 10,€18.70,True,72,46.0
2,5fe6fc8ba789e6e217ef8711,Cocoa Powder - Natural,€17.65,False,45,NaN
3,5fe6fc8ba789e6e217ef8712,Beer - Camerons Cream Ale,€20.19,True,25,60.0
4,5fe6fc8ba789e6e217ef8713,Wine - Tio Pepe Sherry Fino,€17.99,False,27,NaN
...,...,...,...,...,...,...
995,5fe6fc8ba789e6e217ef8ae9,Bread - Rosemary Focaccia,€2.93,False,33,NaN
996,5fe6fc8ba789e6e217ef8aea,Eggs - Extra Large,€21.30,False,40,NaN
997,5fe6fc8ba789e6e217ef8af1,Pasta - Fusili Tri - Coloured,€1.19,False,73,NaN
998,5fe6fc8ba789e6e217ef8af2,Wine - Fume Blanc Fetzer,€8.44,True,94,41.0


Die Daten der Grocery Collection werden uns Korrekt angezeigt. Im nächsten Schritt wollen wir das gleiche mit den Customer-Daten aus der Customer Collection versuchen.

In [18]:
mongo_json = customer_col.find({})
df = pd.DataFrame.from_records(mongo_json)
df

,_id,first_name,last_name,email,last_login,registration_date,address,cart,phone_number
0,5fe60bb2fc13ae64ea000065,Hilde,Brownsett,hbrownsett1@mediafire.com,2019-08-01 22:00:00,2019-08-01 22:00:00,"{'country': 'Indonesia', 'city': 'Suka Makmue'...","[5fe6fc8ba789e6e217ef88bf, 5fe6fc8ba789e6e217e...",NaN
1,5fe60bb2fc13ae64ea000067,Wait,Domek,wdomek3@surveymonkey.com,2019-06-18 22:00:00,2019-06-18 22:00:00,"{'country': 'Brazil', 'city': 'São Miguel dos ...","[5fe6fc8ba789e6e217ef88d1, 5fe6fc8ba789e6e217e...",+55 744 995 7575
2,5fe60bb2fc13ae64ea000068,Bryn,Legat,blegat4@google.co.jp,2020-08-06 22:00:00,2020-08-06 22:00:00,"{'country': 'Indonesia', 'city': 'Muke', 'stre...","[5fe6fc8ba789e6e217ef8764, 5fe6fc8ba789e6e217e...",+62 810 708 7339
3,5fe60bb2fc13ae64ea000069,Aubree,Jessopp,ajessopp5@yellowbook.com,2019-01-02 23:00:00,2019-01-02 23:00:00,"{'country': 'Russia', 'city': 'Nevinnomyssk', ...","[5fe6fc8ba789e6e217ef88be, 5fe6fc8ba789e6e217e...",NaN
4,5fe60bb2fc13ae64ea000066,Bob,Sheridan,bsheridan2@photobucket.com,2020-07-20 22:00:00,2020-07-20 22:00:00,"{'country': 'Netherlands', 'city': 'Haarlem', ...","[5fe6fc8ba789e6e217ef89a0, 5fe6fc8ba789e6e217e...",NaN
...,...,...,...,...,...,...,...,...,...
994,5fe60bb6fc13ae64ea000448,Joshia,McMeyler,jmcmeylerro@cbsnews.com,2019-12-10 23:00:00,2019-12-10 23:00:00,"{'country': 'Afghanistan', 'city': 'Mehtar Lām...","[5fe6fc8ba789e6e217ef89d1, 5fe6fc8ba789e6e217e...",+93 288 375 4524
995,5fe60bb6fc13ae64ea000446,Malvin,Semeniuk,msemeniukrm@homestead.com,2020-04-17 22:00:00,2020-04-17 22:00:00,"{'country': 'Japan', 'city': 'Innoshima', 'str...","[5fe6fc8ba789e6e217ef872e, 5fe6fc8ba789e6e217e...",+81 938 291 8385
996,5fe60bb6fc13ae64ea000445,Milena,Hopfner,mhopfnerrl@purevolume.com,2020-10-10 22:00:00,2020-10-10 22:00:00,"{'country': 'Czech Republic', 'city': 'Černoší...","[5fe6fc8ba789e6e217ef8886, 5fe6fc8ba789e6e217e...",+420 574 478 5225
997,5fe60bb6fc13ae64ea000449,Clovis,Dancey,cdanceyrp@aboutads.info,2019-07-15 22:00:00,2019-07-15 22:00:00,"{'country': 'Indonesia', 'city': 'Gludug', 'st...","[5fe6fc8ba789e6e217ef89c5, 5fe6fc8ba789e6e217e...",NaN


Hierbei ist zu sehen, dass die Embedded Documents nicht als einzelne Spalte interpretiert werden. Diese Daten sollten aber auch korrekt dargestellt werden.

<b>Auslesen der Daten aus dem Embedded Document</b>

In [19]:
mongo_json = customer_col.find({})
md1 = []
mongo_data1 = {}
keys = ["_id","city", "country", "street","zip"]
for key in keys:
    mongo_json = customer_col.find({})
    for doc in mongo_json:
        if key == "_id":
            md1.append(doc["_id"])
        else:
            md1.append(doc["address"].get(key))
    mongo_data1.update({key:list(md1)})
    md1 = []

mongo_dataframe = pd.DataFrame(mongo_data1)
mongo_dataframe

,_id,city,country,street,zip
0,5fe60bb2fc13ae64ea000065,Suka Makmue,Indonesia,0 Merry Crossing,62476
1,5fe60bb2fc13ae64ea000067,São Miguel dos Campos,Brazil,6657 Crescent Oaks Place,77346
2,5fe60bb2fc13ae64ea000068,Muke,Indonesia,72207 Loeprich Center,11922
3,5fe60bb2fc13ae64ea000069,Nevinnomyssk,Russia,8959 Iowa Lane,72362
4,5fe60bb2fc13ae64ea000066,Haarlem,Netherlands,81633 Forest Parkway,49110
...,...,...,...,...,...
994,5fe60bb6fc13ae64ea000448,Mehtar Lām,Afghanistan,60 Elka Junction,92077
995,5fe60bb6fc13ae64ea000446,Innoshima,Japan,790 Esch Junction,37850
996,5fe60bb6fc13ae64ea000445,Černošín,Czech Republic,81 New Castle Junction,87477
997,5fe60bb6fc13ae64ea000449,Gludug,Indonesia,88870 Sutherland Pass,63962


<b>Auslesen der Daten aus einem Array</b>

In [23]:
md1 = []
mongo_data1 = {}
keys = ["_id","cart"]
for key in keys:
    mongo_json = customer_col.find({})
    for doc in mongo_json:
        if key == "_id":
            md1.append(doc["_id"])
        else:
            md1.append(doc["cart"])
    mongo_data1.update({key:list(md1)})
    md1 = []

mongo_dataframe = pd.DataFrame(mongo_data1)
mongo_dataframe

,_id,cart
0,5fe60bb2fc13ae64ea000065,"[5fe6fc8ba789e6e217ef88bf, 5fe6fc8ba789e6e217e..."
1,5fe60bb2fc13ae64ea000067,"[5fe6fc8ba789e6e217ef88d1, 5fe6fc8ba789e6e217e..."
2,5fe60bb2fc13ae64ea000068,"[5fe6fc8ba789e6e217ef8764, 5fe6fc8ba789e6e217e..."
3,5fe60bb2fc13ae64ea000069,"[5fe6fc8ba789e6e217ef88be, 5fe6fc8ba789e6e217e..."
4,5fe60bb2fc13ae64ea000066,"[5fe6fc8ba789e6e217ef89a0, 5fe6fc8ba789e6e217e..."
...,...,...
994,5fe60bb6fc13ae64ea000448,"[5fe6fc8ba789e6e217ef89d1, 5fe6fc8ba789e6e217e..."
995,5fe60bb6fc13ae64ea000446,"[5fe6fc8ba789e6e217ef872e, 5fe6fc8ba789e6e217e..."
996,5fe60bb6fc13ae64ea000445,"[5fe6fc8ba789e6e217ef8886, 5fe6fc8ba789e6e217e..."
997,5fe60bb6fc13ae64ea000449,"[5fe6fc8ba789e6e217ef89c5, 5fe6fc8ba789e6e217e..."


<div style="background-color:  #168b33; padding: 5px;height: 15px;"></div>

### 4.4 <b id="Aufgabenbereich3">Aufgabenbereich 3</b>

Warum zeigt Pandas an, wenn die Daten als JSON Exportiert werden?

In [ ]:
mongo_lm.show_task(8)

Sind CSV Dateien eine Option um den Datenbestand zu betrachten?

In [ ]:
mongo_lm.show_task(9)

In [ ]:
Zeign Sie alle Dokumente an die in der Collection "customer" aus der Datenbank "store_db" sind.

In [ ]:
customer_col = store_db["Customer"]
customer_data = customer_col.find()

customer_dataframe = pd.DataFrame(customer_data)
customer_dataframe.count()

Wie viele Dokumente sind in der Collection enthalten?

In [ ]:
mongo_lm.show_task(10)

In [ ]:
Zeign Sie alle Dokumente an die in der Collection "grocery" aus der Datenbank "store_db" sind.

In [ ]:
customer_col = store_db["Grocery"]
customer_data = customer_col.find()

customer_dataframe = pd.DataFrame(customer_data)
customer_dataframe.count()

Wie viele Dokumente sind in der Collection enthalten?

In [ ]:
mongo_lm.show_task(11)

<div style="background-color: #168b33; padding: 5px; height: 50px;"></div>

## 5 <b id="Queries">Erstellen von Queries mit den MongoDB Operatoren</b>

Da wir nun im Kapitel 2 gesehen haben wie mit der MongoDB interagiert werden kann und in Kapiel 3 betrachtet wurde wie die Daten vorliegen, können nun Anfragen an die Datenbank erstellt werden.

Hierzu werden die MongoDB Operatoren verwendet. Diese ermöglichen es spezifische Anfragen zum vergleichen oder das Suchen von Dokumenten zu implementieren.

### 5.1 <b id="Operatoren-Uebersicht">Übersicht über die Operatoren</b></br>

Diese Operatoren werden verwendet um Ausdrücke und Anfragen an den Datenbestand zu generieren. Sie werden innerhalb einer Query integriert und schränken die Angefragten Daten anhand der definierten Kriterien ein. Einige neue Beispiele sind hier aufgelistet:

<table style="font-size:14px">
	<tbody >
        <tr>
            <td style="width: 200px;text-align: center;" align="center"><b>Operation</b></td>
            <td style="width: 200px;text-align: center;" align="center"><b>Syntax</b></td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Equality</td>
			<td style="width: 200px;text-align: center;" align="center">\$eq</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Not Equal</td>
			<td style="width: 200px;text-align: center;" align="center">\$ne</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Less Than oder Less Than Equal</td>
			<td style="width: 200px;text-align: center;" align="center">\$lt oder \$lte</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Greater Than oder Greater Than Equal</td>
			<td style="width: 200px;text-align: center;" align="center">\$gt oder $gte</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Values in an array</td>
			<td style="width: 200px;text-align: center;" align="center">\$in</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Logisches Und</td>
			<td style="width: 200px;text-align: center;" align="center">\$and</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Logisches Oder</td>
			<td style="width: 200px;text-align: center;" align="center">\$or</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Ob ein Feld existiert</td>
			<td style="width: 200px;text-align: center;" align="center">\$exists</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Nach einem regulären Ausdruck filtern</td>
			<td style="width: 200px;text-align: center;" align="center">\$regex</td>
		</tr>
	</tbody>
</table>

In [ ]:
grocery_col = store_db["Grocery"]

### 5.2 <b id="beispiele-operatoren">Beispiele für die Operatoren</b></br>

<b>Beispiel:</b> Aus der Collection "Grocery" sollen alle Produkte ausgegeben werden die nicht vorrätig sind.

In [ ]:
grocery_col = store_db["Grocery"]

### 5.2 <b id="beispiele-operatoren">Beispiele für die Operatoren</b></br>

<b>Beispiel:</b> Aus der Collection "Grocery" sollen alle Produkte ausgegeben werden die nicht vorrätig sind.

In [ ]:
cursor = grocery_col.find({"in_stock" : { "$eq" : False }})
df = pd.DataFrame(cursor)
df

<b>Beispiel:</b> Aus der Collection "Grocery" sollen alle Produkte ausgegeben werden die häufiger als 99x verkauft wurden.

In [ ]:
cursor = grocery_col.find({"sold":{"$gt":99}})
df = pd.DataFrame(cursor)
df

<b>Beispiel:</b> Aus der Collection "Grocery" sollen alle Produkte ausgegeben werden von dennen weniger als 5 auf Lager sind aber mehr als 3.

In [ ]:
cursor = grocery_col.find({"$and": [{"stock":{"$gt":3}},{"stock":{"$lt":5}}]})
df = pd.DataFrame(cursor)
df

<b>Beispiel:</b> Aus der Collection "Grocery" und der Collection "Customer" sollen alle Produkte ausgegeben werden die der Kunde mit der ID <b>"5fe60bb2fc13ae64ea000067"</b> gekauft hat.

In [ ]:
customer_col = store_db["Customer"]
result = customer_col.find({"_id":ObjectId("5fe60bb2fc13ae64ea000067")})
list_of_products = []
for doc in result:
    list_of_products = doc["cart"].copy()
df = pd.DataFrame(list_of_products)
df

In [ ]:
grocery_col = store_db["Grocery"]
cursor = grocery_col.find({"_id":{"$in":list_of_products}})
df = pd.DataFrame(cursor)
df

### 5.1 <b id="Aufgabenbereich4">Aufgabenbereich 4</b>

<div style="background-color:  #168b33; padding: 5px;height: 15px;"></div>

Zeige alle Produkte an die die Kundin mit dem Namen <b>"Bryn Legat"</b> eingekauft hat.

In [ ]:
customer_col = store_db["Customer"]
result = customer_col.find({"first_name" : "Bryn" })

list_of_products = []
for doc in result:
    list_of_products = doc["cart"].copy()

grocery_col = store_db["Grocery"]
cursor = grocery_col.find({"_id":{"$in":list_of_products}})

df = pd.DataFrame(cursor)
df = df.astype(str)
mongo_lm.show_task(101,df)

Zeigen Sie nur die Produkte an die das Feld "stock" beinhalten. Limitieren Sie die Ausgabe auf die ersten 3 Dokumente.

In [ ]:
customer_col = store_db["Grocery"]
result = customer_col.find({"stock":{"$exists":1}}).limit(3)
df = pd.DataFrame(result)
df = df.astype(str)
mongo_lm.show_task(102,df)

Ein Produkt war wohl sehr beliebt und damit genug davon auf Vorrat vorhanden sind, sollen folgende Anfrage erstellt werden:
Es sollen alle Produkte ausgegeben werden, die mehr als 90x  verkauft wurden. Das Produkt soll aber noch auf Lage sein ("in_stock":True) und es dürfen nicht mehr als 5 Stück im Lager sein.
Verbinden Sie dazu mehrere Operatoren mit dem logischen UND Operator.

In [ ]:
customer_col = store_db["Grocery"]
result = grocery_col.find({"$and": [{"sold":{"$gte":90}},{"in_stock":True},{"stock":{"$lte":5}}]})
df = pd.DataFrame(result)
df
df = df.astype(str)
mongo_lm.show_task(103,df)

<div style="background-color: #168b33; padding: 5px;height: 50px;"></div>

## 6 <b id="Pipeline">Aggregation Pipeline</b>

Aggregationsoperationen werden verwendet um Ergebnisse zu berechnen, Daten welche in vielen Dokumenten vorliegen unter einem Key zu gruppieren oder um ein Ergebniss in einer seperaten Collection abzuspeichern. Weiterhin können viele Operationen zu einer Pipeline angereiht werden. Die Operationen werden dann auf die bereits gruppierten Ergebnisse angewendet was die Möglichkeiten zur Datenverarbeitung umfassend erweitert.
Zu beachten ist aber das die Pipeline den Datenbestand nur anzeigt und Projeziert. Werden beispielsweise Werte in einem Feld abgeändert, muss das Dokument einem Update unterzogen werden um die Änderung zu in der Datenbank festzuhalten.

![](./img/pipeline.jpg)

### 6.1 <b id="Pipeline-Stages-Operatoren">Unterschied zwischen Operatoren und Stages</b>

<b>Stages:</b></br>
    
Im Gegensatz zu den Operatoren werden die Stages in einem Array defniniert der Sequentiell abgearbeitet wird. Jeder Output einer Stage ist somit der neue Input für die nächste Stage. Somit können zu Beginn Gruppierungen oder Sortierungen durchgeführt werden und spezifischere Anfragen an das Ende gestellt werden, um die Performance zu erhöhen.

<table style="font-size:14px;width:900px !important;">
	<tbody >
        <tr>
            <td style="width: 200px;text-align: center;" align="center"><b>Erklärung</b></td>
            <td style="width: 200px;text-align: center;" align="center"><b>Operation</b></td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Fügt neue Felder zu den Dokumenten hinzu. Das Resultat enthällt somit alle Felder die vorher vorhanden waren und die neu hinzugefügten</td>
			<td style="width: 200px;text-align: center;" align="center">\$addFields</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Left outer join. Bindet Dokumente aus einer anderen Collection mit ein. Die zusätzlichen Dokumente werden als Array in die bestehenden Dokumente eingefügt</td>
			<td style="width: 200px;text-align: center;" align="center">\$lookup</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Gruppieren anhand eines Feldes</td>
			<td style="width: 200px;text-align: center;" align="center">\$group</td>
		</tr>
		<tr>
			<td style="width: 200px;text-align: center;" align="center">Filtern von Dokumenten anhand eines Kriteriums</td>
			<td style="width: 200px;text-align: center;" align="center">\$match</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Sortieren der Dokumente anhand eines Kriteriums</td>
			<td style="width: 200px;text-align: center;" align="center">\$sort</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Zerlegt einen Array-Feld innerhalb der Eingabedokumente, um für jedes Element ein Dokument auszugeben</td>
			<td style="width: 200px;text-align: center;" align="center">\$unwind</td>
		</tr>
        <tr>
			<td style="width: 200px;text-align: center;" align="center">Legt die Dokumente in einer neuen Kollektion ab muss (wenn sie verwendet wird) immer die <b style="color:red">letzte Stage sein</b></td>
			<td style="width: 200px;text-align: center;" align="center">\$out</td>
		</tr>
	</tbody>
</table>

### 6.2 <b id="Pipeline-Syntax">Implementieren einer Pipeline (Syntax)</b>

![](./img/syntax_pipeline.jpg)

### 6.3 <b id="Pipeline-Beispiele">Beispiele mit der Aggregation Pipeline</b>

Um ein <b>Beispiel</b> zu demonstrieren verwenden wir wieder die Customer Datenbank

In [ ]:
customer_col = store_db["Customer"]

In der folgen Pipeline werden 4 Stages angereiht und mit verschiedenen Operatoren bearbeitet.</br>

<b>Stages:</b>
<ol>
  <li><b>\$group:</b> Gruppiert alle eingehenden Dokumente mit der ID anhand eines Bestimmten Wertes.</li>
  Vegleichbar mit SQL: <b style="color: green">GROUP by</b>
  Ein neues Feld "count" wird erzeugt welches +1 addiert, wenn ein Dokument gruppiert wurde. </br>
  
  <li><b>\$match:</b> Untersucht die Dokumente anhand bestimmer Konditionen.</li>
  Vegleichbar mit SQL: <b style="color: green">WHERE</b>
  Auf dem neu erzeugten Feld "count" wird der greater than Operator angewendet.
  
  <li><b>\$match:</b> Verhalten ist gleich mit Stage 2 aber ein anderer Operator wird verwendet.</li>
  Der Operator $regex untersucht die Länder anhand eines regulären Ausdruckes.

  <li><b>\$sort:</b> Sortiert die Werte anhand eines Values. 
  Der Wert -1 entspicht absteigend. 1 entspricht    aufsteigend</li>
  Der Operator $regex untersucht die Länder anhand eines regulären Ausdruckes.
</ol>  

Auslesen der Daten aus einem Array

In [50]:
customer_col = store_db["Customer"]
pipeline = [
    #Auflösen des Array -> für jedes Feld im Array entsteht ein neues Dokument
    {"$unwind": "$cart"},
    #Wir zeigen nur en Nachnamen und die ID aus dem Array Cart an
    {"$project": {"cart":1 , "_id":0 ,"last_name":1}},
]

df = pd.DataFrame(customer_col.aggregate(pipeline))
df

,last_name,cart
0,Brownsett,5fe6fc8ba789e6e217ef88bf
1,Brownsett,5fe6fc8ba789e6e217ef88b3
2,Brownsett,5fe6fc8ba789e6e217ef8ab1
3,Brownsett,5fe6fc8ba789e6e217ef8981
4,Brownsett,5fe6fc8ba789e6e217ef8733
...,...,...
12630,Bentick,5fe6fc8ba789e6e217ef882e
12631,Bentick,5fe6fc8ba789e6e217ef8ab8
12632,Bentick,5fe6fc8ba789e6e217ef8acb
12633,Bentick,5fe6fc8ba789e6e217ef8a06


Eine Gruppierung

In [104]:
pipeline = [
    # Die Gruppierung 
    {"$group": {"_id": "$address.country", "Kundenanzahl": {"$sum": 1}}},
]

df = pd.DataFrame(customer_col.aggregate(pipeline))
df

,_id,Kundenanzahl
0,Rwanda,1
1,Burundi,1
2,Liberia,1
3,Ireland,2
4,Sudan,1
...,...,...
123,Bangladesh,4
124,Liechtenstein,1
125,Israel,1
126,Belgium,1


Es sollen alle Länder ausgegeben werden, von denen mehr als 10 Kunden registriert sind und bei dennen der name mit einem P beginnt.

In [98]:
pipeline = [
    # Die Gruppierung wird anhand des Landes durchgeführt und in dem neu erstellten Feld count gezählt
    {"$group": {"_id": "$address.country", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 10}}},
    {"$match": {"_id": {"$regex": '^Po*'}}},
    {"$sort": {"count":1}}
]

df = pd.DataFrame(customer_col.aggregate(pipeline))
df

,_id,count
0,Peru,23
1,Poland,32
2,Portugal,32
3,Philippines,44


Alle Produktdaten von allen Produkten die ein Kunde gekauft hat. (Die Ausgabe ist wegen der Länge auf 1 Dokument limitiert)

In [ ]:
pipeline = [
    {"$lookup": { "from":"Grocery", "localField":"cart", "foreignField":"_id","as": "bought"}},
    {"$project": { "bought": True, "cart": True,"_id": False } },
    {"$limit":1}
]

for doc in customer_col.aggregate(pipeline):
    pprint(doc)

<div style="background-color:  #168b33; padding: 5px;height: 15px;"></div>

### 6.4 <b id="Aufgabenbereich5">Aufgabenbereich 5</b>

Es gibt zwei Produkte mit dem selben Namen, aber eines davon wurde mehr als 50x verkauft. Der Name ist "Cocoa Powder - Natural". Finden Sie dieses Produkt.

In [ ]:
grocery_col = store_db["Grocery"]
pipeline = [
    {"$match" : {"product":"Cocoa Powder - Natural"}},
    {"$match" : {"sold": {"$gt" : 50}}}
]

df = pd.DataFrame(grocery_col.aggregate(pipeline))
df = df.astype(str)
mongo_lm.show_task(104,df)

Es soll die Gesamten Einnahmen von allen Produkten berechnet werden (Price x Sold)</br>

<b> Hierbei ist zu beachten das der Preis in den Dokumenten als String eingetragen ist.</b>
Entfernen Sie dazu das $ Symbol und wandeln Sie dann die Preise in decimal um und updaten Sie jedes Dokument in einer For-Schleife.

In [ ]:
pipeline = [
    {"$addFields": {"price" : {"$trim": {"input": "$price", "chars": "€"}}}},
    {"$addFields": {"price" : {"$convert": {"input": "$price", "to": "double"}}}},
    {"$group": {"_id": None, "count": { "$sum": { "$multiply": [ "$price", "$sold" ] } }}},
    {"$out" : "gains_collection"}
]

grocery_col.aggregate(pipeline)
new_col = store_db["gains_collection"]
df = pd.DataFrame(new_col.find())
df = df.astype(str)
mongo_lm.show_task(105,df)

Es sollen alle Kunden aus den Städten gezählt werden, in dennen mehr als 3 wohnen.

In [99]:
customer_col = store_db["Customer"]
pipeline = [
    {"$group": {"_id": "$address.city","count": { "$sum": 1 }}},
    {"$sort": {"count":1}},
    {"$match": {"count":{"$gt":2}}},
]

df = pd.DataFrame(customer_col.aggregate(pipeline))
df = df.astype(str)
mongo_lm.show_task(106,df)

,_id,count
0,Stockholm,3


Output()

Output()

Die Anzahl aller Kunden die mehr als 25 Produkte gekauft haben

In [102]:
customer_col = store_db["Customer"]
pipeline = [
    {"$unwind": "$cart"},
    {"$project": {"cart":1 , "_id":1 ,"last_name":1}},
    {"$group" : {"_id":"$_id", "count": { "$sum": 1 }}},
    {"$match" : {"count":{"$gt" : 24}}},
    {"$group" : {"_id":None , "count" :{ "$sum": 1 }}}
]

df = pd.DataFrame(customer_col.aggregate(pipeline))
df = df.astype(str)
mongo_lm.show_task(107,df)

,_id,count
0,None,41


Output()

Output()

<div style="background-color: #168b33; padding: 5px;height: 50px;"></div>

## 7 <b id="Map">Map-Reduce</b>

<div style="background-color:  #168b33; padding: 5px;height: 15px;"></div>

### 7.1 <b id="Aufgabenbereich6">Aufgabenbereich 6</b>

<div style="background-color: #168b33; padding: 5px;height: 50px;"></div>

## 8 <b id="Abschlussaufgabe">Abschlussaufgabe</b>

Erstellen Sie mit Hilfe der Aggregation Pipeline eine neue Collection "Transactions" die alle Einkäufe auflistet.

### 8.1 <b id="Aufgabenbereich7">Aufgabenbereich 7</b>